# 年間配当額の推移

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hiohiohio/jquants-api-client-python/blob/feature/add-sample-notebooks/examples/20220825-004-annual-dividend-history.ipynb)

このノートブックでは J-Quants APIから取得できるデータを利用して特定銘柄についての年間配当額の推移を表示します。


----

**このノートブックはGoogle Driveを使用します。**

- Google Drive の以下のファイルにリフレッシュトークンが書き込まれていることを想定しています。
    - `MyDrive/drive_ws/secret/jquantsapi-key.txt`
- Google Drive の以下のフォルダーにデータを書き込みます。
    - `MyDrive/drive_ws/marketdata`

In [ ]:
# 年間配当額の推移を表示する銘柄コードを指定します (e.g. 86970)
TARGET_SYMBOL = "86970"  # 日本取引所グループ

In [ ]:
# 必要なモジュールをインストールします。
! python -m pip install jquants-api-client japanize-matplotlib

In [ ]:
# Google drive をマウントします。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from datetime import datetime

import japanize_matplotlib
import jquantsapi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# pandas の表示制限を調整します
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 2000)

In [ ]:
# プロット用の設定をします
sns.set(rc={'figure.figsize': (15, 10)})
sns.set(font_scale=1.5)
sns.set_style('whitegrid')

japanize_matplotlib.japanize()

In [ ]:
# 一度取得したデータは Google Drive 上に保存して再利用します。
# 保存先ディレクトリを指定します。
STORAGE_DIR_PATH = "/content/drive/MyDrive/drive_ws/marketdata"
os.makedirs(STORAGE_DIR_PATH, exist_ok=True)
STORAGE_DIR_PATH

In [ ]:
# J-Quants APIのトークンを保存してあるファイルを指定します
REFRESH_TOKEN_FILE_PATH = "/content/drive/MyDrive/drive_ws/secret/jquantsapi-key.txt"

In [ ]:
def get_refresh_token(refresh_token_file_path: str = REFRESH_TOKEN_FILE_PATH):
    with open(refresh_token_file_path, "r") as f:
        refresh_token = f.read()
    return refresh_token.rstrip().lstrip()

In [ ]:
# ファイルからリフレッシュトークンを読み込みます
refresh_token = get_refresh_token()

In [ ]:
# J-Quants APIクライアントを初期化します
jqapi = jquantsapi.Client(refresh_token=refresh_token)

In [ ]:
# 銘柄情報を取得します
now = pd.Timestamp.now(tz="Asia/Tokyo")
if now.hour < 22:
    # データ更新時間前の場合は日付を1日ずらします。
    now -= pd.Timedelta(1, unit="D")
list_file = f"{STORAGE_DIR_PATH}/list_{now.strftime('%Y%m%d')}.csv.gz"
if not os.path.isfile(list_file):
    df_list = jqapi.get_list()
    df_list.to_csv(list_file, compression="gzip", index=False)
    print(f"save file: {list_file}")

# ファイルからデータを読み込みます
print(f"file exists: {list_file}, loading")
df_list = pd.read_csv(list_file, dtype="str")

In [ ]:
# 対象銘柄の財務情報を取得します
df_s_org = jqapi.get_fins_statements(code=TARGET_SYMBOL)

In [ ]:
# 作業用にコピーします
df_s = df_s_org.copy()

# 年度末情報に絞り込みます
df_s = df_s.loc[df_s["TypeOfCurrentPeriod"] == "FY"]
# float64にするために"-"および""をnp.nanに置き換えます
df_s.replace({"－": np.nan, "": np.nan}, inplace=True)
df_s.loc[:, "ResultDividendPerShareAnnual"] = df_s["ResultDividendPerShareAnnual"].astype(np.float64)
df_s.loc[:, "ForecastDividendPerShareAnnual"] = df_s["ForecastDividendPerShareAnnual"].astype(np.float64)
df_s.loc[:, "EarningsPerShare"] = df_s["EarningsPerShare"].astype(np.float64)

# 日付型に変換します
df_s.loc[:, "DisclosedDate"] = pd.to_datetime(
    df_s["DisclosedDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentPeriodEndDate"] = pd.to_datetime(
    df_s["CurrentPeriodEndDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentFiscalYearStartDate"] = pd.to_datetime(
    df_s["CurrentFiscalYearStartDate"], format="%Y-%m-%d"
)
df_s.loc[:, "CurrentFiscalYearEndDate"] = pd.to_datetime(
    df_s["CurrentFiscalYearEndDate"], format="%Y-%m-%d"
)
# 開示順に並べ替えます
df_s.sort_values("DisclosedUnixTime", inplace=True)

# 配当性向を算出します
df_s["配当性向"] = df_s["ResultDividendPerShareAnnual"] / df_s["EarningsPerShare"]

In [ ]:
# プロット用に配当額が存在するレコードのみに絞り込みます
df_plot = df_s.loc[df_s["ResultDividendPerShareAnnual"].notna()].set_index("DisclosedDate")
# インデックスの表記を変更します
df_plot.index = df_plot.index.strftime("%Y-%m-%d")

# 項目名を日本語にします
df_plot.index.name = "開示日"
df_plot.rename(columns={
    "ResultDividendPerShareAnnual": "配当実績_合計",
    "EarningsPerShare": "1株当たり当期純利益",
}, inplace=True)

# 配当額と1株あたり利益をプロットします
ax = df_plot[["配当実績_合計", "1株当たり当期純利益"]].plot(kind="bar")
# 配当性向をプロットします
bx = df_plot[["配当性向"]].plot(linestyle='-', marker='o', ax=ax, use_index=False, secondary_y=True, color="black")

# 棒グラフに数字を表記します
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

# 銘柄名を取得します
name = df_list.loc[df_list["Code"] == TARGET_SYMBOL, "CompanyName"].iat[0]

# タイトルを設定します
ax.set_title(f"{name}({TARGET_SYMBOL}) の年間配当推移")